<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания № 12


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Описание задачи:

Создать базовый класс Item в C#, который будет представлять информацию о
товарах, которые могут быть заказаны или возвращены. На основе этого класса
разработать 2-3 производных класса, демонстрирующих принципы наследования и
полиморфизма. В каждом из классов должны быть реализованы новые атрибуты и
методы, а также переопределены некоторые методы базового класса для
демонстрации полиморфизма.

Требования к базовому классу Item:

• Атрибуты: ID товара (ItemId), Название (Name), Цена (Price).

• Методы:

o GetDetails(): метод для получения детальной информации о товаре.

o CalculateDiscount(): метод для расчета скидки на товар.

o ApplyDiscount(decimal discount): метод для применения скидки к цене
товара.

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) создайте явную реализации интерфейса и управление зависимостями 


<h2 style="color:DodgerBlue">Реализация:</h2>

----